# AI-powered Mario

Authors: [Yuansong Feng](https://github.com/YuansongFeng), [Suraj Subramanian](https://github.com/suraj813), [Howard Wang](https://github.com/hw26), [Steven Guo](https://github.com/GuoYuzhang).  


## Welcome!
This tutorial walks you through the fundamentals of Deep Reinforcement Learning. At the end, you will implement an AI-powered Mario (using [Double Deep Q-Networks](https://arxiv.org/pdf/1509.06461.pdf)) that can play the game by itself. 

Although no prior knowledge of RL is necessary for this tutorial, you can familiarize yourself with these RL [concepts](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html), and have this handy [cheatsheet](https://colab.research.google.com/drive/1eN33dPVtdPViiS1njTW_-r-IYCDTFU7N) as your companion. The full code is available [here](https://github.com/yuansongFeng/MadMario/).

## Setup

In [15]:
# Mario game environment
#!pip install gym-super-mario-bros==7.3.0 opencv-python

import os
import copy
import torch
from torch import nn
from pathlib import Path
from collections import deque
import random, datetime, numpy as np, cv2 
# Gym is an OpenAI toolkit for RL
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack, GrayScaleObservation, TransformObservation

#NES Emulator for OpenAI Gym
from nes_py.wrappers import JoypadSpace

# Super Mario environment for OpenAI Gym
import gym_super_mario_bros

## RL Definitions

**Environment** 
The world that an agent interacts with and learns from.

**Action** $a$ : 
How the Agent responds to the Environment. The set of all possible Actions is called *action-space*.

**State** $s$ :
The current characteristic of the Environment. The set of all possible States the Environment can be in is called *state-space*.

**Reward** $r$ :
Reward is the key feedback from Environment to Agent. It is what drives the Agent to learn and to change its future action. An aggregation of rewards over multiple time steps is called **Return**.
 
 **Optimal Action-Value function** $Q^*(s,a)$ :
Gives the expected return if you start in state $s$, take an arbitrary action $a$, and then for each future time step take the action that maximizes returns. $Q$ can be said to stand for the "quality" of the action in a state. We try to approximate this function. 


# Initialize Environment 
In Mario, the environment consists of tubes, mushrooms and other components. 

When Mario makes an action, the environment responds with the changed (next) state, reward and other info.

In [2]:
# Initialize Super Mario environment
env = gym_super_mario_bros.make('SuperMarioBros-v3')

# Limit the action-space to
#   0. walk right 
#   1. jump right
env = JoypadSpace(
    env,
    [['right'],
    ['right', 'A']]
)

env.reset()
next_state, reward, done, info = env.step(action=0)
print(f'{next_state.shape},\n {reward},\n {done},\n {info}')

(240, 256, 3),
 0,
 False,
 {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'x_pos_screen': 40, 'y_pos': 79}


# Preprocess Environment
Environment data is returned to the agent in `next_state`. As you saw above, each state is represented by a `[3, 240, 256]` size array. Often that is more information than our agent needs; for instance, Mario's actions do not depend on the color of the pipes or the sky!

We use **Wrappers** to preprocess environment data before sending it to the agent. 

`GrayScaleObservation` is a common wrapper to transform an RGB image to grayscale; doing so reduces the size of the state representation without losing useful information. Now the size of each state: `[1, 240, 256]`

`ResizeObservation` downsamples each observation into a square image. New size: `[1, 84, 84]`

`SkipFrame` is a custom wrapper that inherits from `gym.Wrapper` and implements the `step()` function. Because consecutive frames don't vary much, we can skip n-intermediate frames without losing much information. The n-th frame aggregates rewards accumulated over each skipped frame.

`FrameStack` is a wrapper that allows us to squash consecutive frames of the environment into a single observation point to feed to our learning model. This way, we can identify if Mario was landing or jumping based on the direction of his movement in the previous several frames.


In [3]:
class ResizeObservation(gym.ObservationWrapper):
    def __init__(self, env, shape):
        super().__init__(env)
        if isinstance(shape, int):
            self.shape = (shape, shape)
        else:
            self.shape = tuple(shape)

        obs_shape = self.shape + self.observation_space.shape[2:]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def observation(self, observation):
        observation = cv2.resize(observation, self.shape, interpolation=cv2.INTER_AREA)
        return observation


class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        """Return only every `skip`-th frame"""
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        """Repeat action, and sum reward"""
        total_reward = 0.0
        done = False
        for i in range(self._skip):
            # Accumulate reward and repeat the same action
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, info


# Apply Wrappers to environment
env = SkipFrame(env, skip=4)
env = GrayScaleObservation(env, keep_dim=False)
env = ResizeObservation(env, shape=84)
env = TransformObservation(env, f=lambda x: x / 255.)
env = FrameStack(env, num_stack=4)

After applying the above wrappers to the environment, the final wrapped state consists of 4 gray-scaled consecutive frames stacked together, as shown above in the image on the left. Each time Mario makes an action, the environment responds with a state of this structure. The structure is represented by a 3-D array of size `[4, 84, 84]`.


![picture](https://drive.google.com/uc?id=1zZU63qsuOKZIOwWt94z6cegOF2SMEmvD)

# Agent
We create a class `Mario` to represent our agent in the game. Mario should be able to:

- **Act** according to the optimal action policy based on the current state (of the environment).

- **Remember** experiences. Experience = (current state, current action, reward, next state). Mario *caches* and later *recalls* his experiences to update his action policy.

- **Learn** a better action policy over time

In [4]:
class Mario:
    def __init__():
        pass

    def act(self, state):
        """Given a state, choose an epsilon-greedy action"""
        pass

    def cache(self, experience):
        """Add the experience to memory"""
        pass

    def recall(self):
        """Sample experiences from memory"""
        pass

    def learn(self):
        """Update online action value (Q) function with a batch of experiences"""
        pass

In the following sections, we will populate Mario's parameters and define his functions.

# Act

For any given state, an agent can choose to do the most optimal action (***exploit***) or a random action (***explore***). 

Mario randomly explores with a chance of `self.exploration_rate`; when he chooses to exploit, he relies on `MarioNet` (implemented in `Learn` section) to provide the most optimal action.

In [5]:
class Mario: 
  def __init__(self, state_dim, action_dim, save_dir):
    self.state_dim = state_dim
    self.action_dim = action_dim
    self.save_dir = save_dir

    self.use_cuda = torch.cuda.is_available()
    
    # Mario's DNN to predict the most optimal action - we implement this in the Learn section
    self.net = MarioNet(self.state_dim, self.action_dim).float()
    if self.use_cuda:
      self.net = self.net.to(device='cuda')

    self.init_exploration_rate = 0.7
    self.exploration_rate = 0.7
    self.exploration_rate_decay = 0.999999
    self.exploration_rate_min = 0.1
    self.curr_step = 0

    self.save_every = 5e5   # no. of experiences between saving Mario Net
    

  def act(self, state):
    """
    Given a state, choose an epsilon-greedy action and update value of step.

    Inputs:
    state(LazyFrame): A single observation of the current state, dimension is (state_dim)
    Outputs:
    action_idx (int): An integer representing which action Mario will perform
    """
    # EXPLORE
    if np.random.rand() < self.exploration_rate:
        action_idx = np.random.randint(self.action_dim)

    # EXPLOIT
    else:
        state = np.array(state)
        state = torch.FloatTensor(state).cuda() if self.use_cuda else torch.FloatTensor(state)
        state = state.unsqueeze(0)
        action_values = self.net(state, model='online')[0]
        action_idx = torch.argmax(action_values, axis=1).item()

    # decrease exploration_rate
    if( self.curr_step < self.burnin):
        self.exploration_rate = 1
    elif self.curr_step == self.burnin:
        self.exploration_rate = self.init_exploration_rate
    else:
        self.exploration_rate *= self.exploration_rate_decay
    self.exploration_rate = max(self.exploration_rate_min, self.exploration_rate)

    # increment step
    self.curr_step += 1
    return action_idx


# Cache and Recall

These two functions serve as Mario's "memory" process. 

`cache()`: Each time Mario performs an action, he stores the `experience` to his memory. His experience includes the current *state*, *action* performed, *reward* from the action, the *next state*, and whether the game is *done*.

`recall()`: Mario randomly samples a batch of experiences from his memory, and uses that to learn the game. 

In [6]:
class Mario(Mario): # subclassing for continuity
  def __init__(self, state_dim, action_dim, save_dir):
    super().__init__(state_dim, action_dim, save_dir)
    self.memory = deque(maxlen=10000)
    self.batch_size = 32
    

  def cache(self, state, y_pos, next_state, action, reward, done):
    """
    Store the experience to self.memory (replay buffer)

    Inputs:
    state (LazyFrame),
    y_pos (float),
    next_state (LazyFrame),
    action (int),
    reward (float),
    done(bool))
    """
    state = np.array(state)
    next_state = np.array(next_state)
    
    state = torch.FloatTensor(state).cuda() if self.use_cuda else torch.FloatTensor(state)
    y_pos = torch.FloatTensor([y_pos]).cuda() if self.use_cuda else torch.FloatTensor(state)
    next_state = torch.FloatTensor(next_state).cuda() if self.use_cuda else torch.FloatTensor(next_state)
    action = torch.LongTensor([action]).cuda() if self.use_cuda else torch.LongTensor([action])
    reward = torch.DoubleTensor([reward]).cuda() if self.use_cuda else torch.DoubleTensor([reward])
    done = torch.BoolTensor([done]).cuda() if self.use_cuda else torch.BoolTensor([done])

    self.memory.append( (state, y_pos, next_state, action, reward, done,) )

  
  def recall(self):
    """
    Retrieve a batch of experiences from memory
    """
    batch = random.sample(self.memory, self.batch_size)
    state, y_pos, next_state, action, reward, done = map(torch.stack, zip(*batch))
    return state, y_pos.squeeze(), next_state, action.squeeze(), reward.squeeze(), done.squeeze()

# Learn

Mario uses the [DDQN algorithm](https://arxiv.org/pdf/1509.06461) under the hood. DDQN uses two ConvNets - $Q_{online}$ and $Q_{target}$ - that independently approximate the optimal action-value function. 

In our implementation, we share feature generator `features` across $Q_{online}$ and $Q_{target}$, but maintain separate FC classifiers for each. $\theta_{target}$ (the parameters of $Q_{target}$) is frozen to prevent updation by backprop. Instead, it is periodically synced with $\theta_{online}$ (more on this later).

### Neural Network

In [14]:
class MarioNet(nn.Module):
  '''mini cnn structure
  input -> (conv2d + relu) x 3 -> flatten -> (dense + relu) x 2 -> output
  '''
  def __init__(self, input_dim, output_dim):
      super().__init__()
      c, h, w = input_dim

      if h != 84:
          raise ValueError(f"Expecting input height: 32, got: {h}")
      if w != 84:
          raise ValueError(f"Expecting input width: 32, got: {w}")

#       self.online = nn.Sequential(
#           nn.Conv2d(in_channels=c, out_channels=32, kernel_size=5, stride=2, padding=1),
#           nn.ReLU(),
#           nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1),
#           nn.ReLU(),
#           #nn.MaxPool2d(3),
          
#           nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
#           nn.ReLU(),
#           nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
#           nn.MaxPool2d(2),
          
#           nn.Flatten(),
#           nn.Linear(18496, 128),
#           nn.ReLU(),
#           nn.Linear(128, output_dim)
#       )

      self.online_features = self.feature_extraction(c, h, w, output_dim)
      self.online_td_est  = self.td_est(output_dim)
      self.online_aux = self.y_pos_est()

      self.target_features = copy.deepcopy(self.online_features)
      self.target_td_est = copy.deepcopy(self.online_td_est)
        
      # Q_target parameters are frozen.
      for p in self.target_features.parameters():
          p.requires_grad = False
      for p in self.target_td_est.parameters():
          p.requires_grad = False

        
  def feature_extraction(self, c, h, w, output_dim):
    return nn.Sequential(
          nn.Conv2d(in_channels=c, out_channels=32, kernel_size=5, stride=2, padding=1),
          nn.ReLU(),
          nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1),
          nn.ReLU(),
          #nn.MaxPool2d(3),
          
          nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1),
          nn.ReLU(),
          nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
          nn.MaxPool2d(2),
          nn.Flatten()
    )

  def td_est(self, output_dim):
    return nn.Sequential(
        nn.Linear(18496, 128),
        nn.ReLU(),
        nn.Linear(128, output_dim)
    )

  def y_pos_est(self):
    return nn.Sequential(
        nn.Linear(18496, 32),
        nn.ReLU(),
        nn.Linear(32, 1)
    )

  def forward(self, input, model):
    if model == 'online':
        features = self.online_features(input)
        return self.online_td_est(features), self.online_aux(features)
    elif model == 'target':
        features = self.target_features(input)
        return self.target_td_est(features), []
    
    

## TD Estimate & TD Target

Two values are involved in learning: 

<br/>

**TD Estimate** - the predicted optimal $Q^*$  for a given state $s$ 

$$
{TD}_e = Q_{online}^*(s,a)
$$
<br/>  


**TD Target** - aggregation of current reward and the estimated $Q^*$ in the next state $s'$

$$
a' = argmax_{a} Q_{online}(s', a)
$$
$$
{TD}_t = r + \gamma Q_{target}^*(s',a')
$$
<br/>

Because we don’t know what next action $a'$ will be, we use the action $a'$ maximizes $Q_{online}$ in the next state $s'$.

Notice we use the [@torch.no_grad()](https://pytorch.org/docs/stable/generated/torch.no_grad.html#no-grad) decorator on `td_target()` to disable gradient calculations here (because we don't need to backpropagate on $\theta_{target}$).


In [8]:
class Mario(Mario):
  def __init__(self, state_dim, action_dim, save_dir):
    super().__init__(state_dim, action_dim, save_dir)
    self.gamma = 0.95
    
  def td_and_aux_estimate(self, state, action):
    outs = self.net(state, model='online')
    current_Q = outs[0][np.arange(0, self.batch_size), action] # Q_online(s,a)
    y_pos_est = outs[1]
    return current_Q, y_pos_est

  @torch.no_grad()
  def td_target(self, reward, next_state, done):
    next_state_Q = self.net(next_state, model='online')[0]
    best_action = torch.argmax(next_state_Q, axis=1)
    next_Q = self.net(next_state, model='target')[0][np.arange(0, self.batch_size), best_action]
    return (reward + (1 - done.float()) * self.gamma * next_Q).float()

## Updating the model

As Mario samples inputs from his replay buffer, we compute $TD_t$ and $TD_e$ and backpropagate this loss down $Q_{online}$ to update its parameters $\theta_{online}$ ($\alpha$ is the learning rate `lr` passed to the `Adam optimizer`)
$$
\theta_{online} \leftarrow \theta_{online} + \alpha \nabla(TD_e - TD_t)
$$ 
<br/>
$\theta_{target}$ does not update through backpropagation.
Instead, we periodically copy $\theta_{online}$ to $\theta_{target}$
$$
\theta_{target} \leftarrow \theta_{online}
$$

In [9]:
class Mario(Mario):
    def __init__(self, state_dim, action_dim, save_dir):
      super().__init__(state_dim, action_dim, save_dir)
      self.optimizer = torch.optim.Adam(self.net.parameters(), lr=0.00025)
      self.loss_fn = torch.nn.SmoothL1Loss()

    def update_Q_online(self, td_estimate, td_target, y_pos_est, y_pos) :
      loss = self.loss_fn(td_estimate, td_target) + (0.2 * self.loss_fn(y_pos_est, y_pos))
      self.optimizer.zero_grad()
      loss.backward()
      self.optimizer.step()
      return loss.item()
      
    def sync_Q_target(self):
      self.net.target_features.load_state_dict(self.net.online_features.state_dict())
      self.net.target_td_est.load_state_dict(self.net.online_td_est.state_dict())

## Save checkpoint

In [10]:
class Mario(Mario):
    def save(self):
        save_path = self.save_dir / f"mario_net_{int(self.curr_step // self.save_every)}.chkpt"
        torch.save(
            dict(
                model=self.net.state_dict(),
                exploration_rate=self.exploration_rate
            ),
            save_path
        )
        print(f"MarioNet saved to {save_path} at step {self.curr_step}")
        
    def load(self, load_path):
        if not load_path.exists():
            raise ValueError(f"{load_path} does not exist")

        ckp = torch.load(load_path, map_location=('cuda' if self.use_cuda else 'cpu'))
        exploration_rate = ckp.get('exploration_rate')
        state_dict = ckp.get('model')

        print(f"Loading model at {load_path} with exploration rate {exploration_rate}")
        self.net.load_state_dict(state_dict)
        self.exploration_rate = exploration_rate

## Putting it all together


In [11]:
class Mario(Mario):
    def __init__(self, state_dim, action_dim, save_dir):
        super().__init__(state_dim, action_dim, save_dir)
        self.burnin = 5e4  # min. experiences before training
        self.learn_every = 3   # no. of experiences between updates to Q_online
        self.sync_every = 1e4   # no. of experiences between Q_target & Q_online sync
        

    def learn(self):
      if self.curr_step % self.sync_every == 0:
          self.sync_Q_target()

      if self.curr_step % self.save_every == 0:
          self.save()

      if self.curr_step < self.burnin:
          return None, None

      if self.curr_step % self.learn_every != 0:
          return None, None

      # Sample from memory
      state, y_pos, next_state, action, reward, done = self.recall()

      # Get TD Estimate
      td_est, y_pos_est = self.td_and_aux_estimate(state, action)

      # Get TD Target
      td_tgt = self.td_target(reward, next_state, done)

      # Backpropagate loss through Q_online
      loss = self.update_Q_online(td_est, td_tgt, y_pos_est, torch.reshape(y_pos, y_pos_est.shape))

      return (td_est.mean().item(), loss)


# Logging 

In [12]:
import numpy as np
import time, datetime
import matplotlib.pyplot as plt

class MetricLogger():
    def __init__(self, save_dir):
        self.save_log = save_dir / "log"
        with open(self.save_log, "w") as f:
            f.write(
                f"{'Episode':>8}{'Step':>8}{'Epsilon':>10}{'MeanReward':>15}"
                f"{'MeanLength':>15}{'MeanLoss':>15}{'MeanQValue':>15}"
                f"{'TimeDelta':>15}{'Time':>20}\n"
            )
        self.ep_rewards_plot = save_dir / "reward_plot.jpg"
        self.ep_lengths_plot = save_dir / "length_plot.jpg"
        self.ep_avg_losses_plot = save_dir / "loss_plot.jpg"
        self.ep_avg_qs_plot = save_dir / "q_plot.jpg"

        # History metrics
        self.ep_rewards = []
        self.ep_lengths = []
        self.ep_avg_losses = []
        self.ep_avg_qs = []

        # Moving averages, added for every call to record()
        self.moving_avg_ep_rewards = []
        self.moving_avg_ep_lengths = []
        self.moving_avg_ep_avg_losses = []
        self.moving_avg_ep_avg_qs = []

        # Current episode metric
        self.init_episode()

        # Timing
        self.record_time = time.time()


    def log_step(self, reward, loss, q):
        self.curr_ep_reward += reward
        self.curr_ep_length += 1
        if loss:
            self.curr_ep_loss += loss
            self.curr_ep_q += q
            self.curr_ep_loss_length += 1

    def log_episode(self):
        "Mark end of episode"
        self.ep_rewards.append(self.curr_ep_reward)
        self.ep_lengths.append(self.curr_ep_length)
        if self.curr_ep_loss_length == 0:
            ep_avg_loss = 0
            ep_avg_q = 0
        else:
            ep_avg_loss = np.round(self.curr_ep_loss / self.curr_ep_loss_length, 5)
            ep_avg_q = np.round(self.curr_ep_q / self.curr_ep_loss_length, 5)
        self.ep_avg_losses.append(ep_avg_loss)
        self.ep_avg_qs.append(ep_avg_q)

        self.init_episode()

    def init_episode(self):
        self.curr_ep_reward = 0.0
        self.curr_ep_length = 0
        self.curr_ep_loss = 0.0
        self.curr_ep_q = 0.0
        self.curr_ep_loss_length = 0

    def record(self, episode, epsilon, step):
        mean_ep_reward = np.round(np.mean(self.ep_rewards[-100:]), 3)
        mean_ep_length = np.round(np.mean(self.ep_lengths[-100:]), 3)
        mean_ep_loss = np.round(np.mean(self.ep_avg_losses[-100:]), 3)
        mean_ep_q = np.round(np.mean(self.ep_avg_qs[-100:]), 3)
        self.moving_avg_ep_rewards.append(mean_ep_reward)
        self.moving_avg_ep_lengths.append(mean_ep_length)
        self.moving_avg_ep_avg_losses.append(mean_ep_loss)
        self.moving_avg_ep_avg_qs.append(mean_ep_q)


        last_record_time = self.record_time
        self.record_time = time.time()
        time_since_last_record = np.round(self.record_time - last_record_time, 3)

        print(
            f"Episode {episode} - "
            f"Step {step} - "
            f"Epsilon {epsilon} - "
            f"Mean Reward {mean_ep_reward} - "
            f"Mean Length {mean_ep_length} - "
            f"Mean Loss {mean_ep_loss} - "
            f"Mean Q Value {mean_ep_q} - "
            f"Time Delta {time_since_last_record} - "
            f"Time {datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')}"
        )

        with open(self.save_log, "a") as f:
            f.write(
                f"{episode:8d}{step:8d}{epsilon:10.3f}"
                f"{mean_ep_reward:15.3f}{mean_ep_length:15.3f}{mean_ep_loss:15.3f}{mean_ep_q:15.3f}"
                f"{time_since_last_record:15.3f}"
                f"{datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S'):>20}\n"
            )

        for metric in ["ep_rewards", "ep_lengths", "ep_avg_losses", "ep_avg_qs"]:
            plt.plot(getattr(self, f"moving_avg_{metric}"))
            plt.savefig(getattr(self, f"{metric}_plot"))
            plt.clf()


# Let's play!

In [13]:
use_cuda = torch.cuda.is_available()

if use_cuda:
    torch.cuda.empty_cache()

import gc
gc.collect()

    
print(f"Using CUDA: {use_cuda}")
print()

save_dir = Path('checkpoints') / datetime.datetime.now().strftime('%Y-%m-%dT%H-%M-%S')
save_dir.mkdir(parents=True)

mario = Mario(state_dim=(4, 84, 84), action_dim=env.action_space.n, save_dir=save_dir)
 
# path = Path('checkpoints') / 'checkpoints/2022-04-13T20-59-01/mario_net_11.chkpt'
# mario.load(path)

logger = MetricLogger(save_dir)

episodes = 40000

### for Loop that train the model num_episodes times by playing the game
for e in range(episodes):

    state = env.reset()

    # Play the game!
    while True:

        # Run agent on the state
        action = mario.act(state)

        # Agent performs action
        next_state, reward, done, info = env.step(action)

        # Remember
        mario.cache(state, info['y_pos'], next_state, action, reward, done)

        # Learn
        q, loss = mario.learn()

        # Logging
        logger.log_step(reward, loss, q)

        # Update state
        state = np.array(next_state)

        # Check if end of game
        if done or info['flag_get']:
            break

    logger.log_episode()

    if e % 20 == 0:
        logger.record(
            episode=e,
            epsilon=mario.exploration_rate,
            step=mario.curr_step
        )

Using CUDA: True



/tmp/ipykernel_35668/3124848373.py:28: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  done = torch.BoolTensor([done]).cuda() if self.use_cuda else torch.BoolTensor([done])
/home/daniel/.local/share/virtualenvs/CM50270_CW2-3mDn6YTX/lib/python3.8/site-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Episode 0 - Step 1635 - Epsilon 1 - Mean Reward 2023.0 - Mean Length 1635.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 10.137 - Time 2022-04-27T16:23:17
Episode 20 - Step 18077 - Epsilon 1 - Mean Reward 1860.095 - Mean Length 860.81 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 104.008 - Time 2022-04-27T16:25:01
Episode 40 - Step 36871 - Epsilon 1 - Mean Reward 1885.78 - Mean Length 899.293 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 120.513 - Time 2022-04-27T16:27:02
Episode 60 - Step 53725 - Epsilon 0.6973980465433149 - Mean Reward 1804.393 - Mean Length 880.738 - Mean Loss 0.475 - Mean Q Value 0.153 - Time Delta 125.829 - Time 2022-04-27T16:29:08
Episode 80 - Step 68874 - Epsilon 0.6869127794719724 - Mean Reward 1815.235 - Mean Length 850.296 - Mean Loss 1.186 - Mean Q Value 1.087 - Time Delta 175.646 - Time 2022-04-27T16:32:03


KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

In [ ]:
def render_episode(mario):
    
    done = True
    for step in range(5000):
        if done:
            state = env.reset()
        
        # Run agent on the state
        action = mario.act(state)

        # Agent performs action
        next_state, reward, done, info = env.step(action)
        
        env.render()
        
# mario = Mario(state_dim=(4, 84, 84), action_dim=env.action_space.n, save_dir=save_dir)
# path = Path('checkpoints') / 'mario_net_12.chkpt'
# mario.load(path)
mario.exploration_rate = 0
render_episode(mario)